First, let's import basic dependencies

In [1]:
!pip install -U langgraph langsmith
!pip install -U "langchain[google-genai]"
!pip install -U langchain-tavily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requ

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 78.1 MB/s eta 0:00:00


In [2]:
import os
from langchain.chat_models import init_chat_model
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages

Create a new class which will help us in defining a node

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

creating a basic chatbot

In [5]:
llm = init_chat_model("google_genai:gemini-2.0-flash")
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


 The first argument is the unique node name

 The second argument is the function or object that will be called whenever the node is used.

some more graph builder functions

In [8]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")

graph = graph_builder.compile()


The chatbot feed

In [7]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

The user feed

In [ ]:
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

our basic chatbot has been built. now, we will walk upon the PS.

integrating tavily

In [10]:

%env TAVILY_API_KEY=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

env: TAVILY_API_KEY=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


In [11]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
tools = [tool]
tool.invoke("Hello what's the admission process for IIT Guwahati")

{'query': "Hello what's the admission process for IIT Guwahati",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'IIT Guwahati (IITG) Admission 2025: Dates, Application ... - Careers360',
   'url': 'https://www.careers360.com/university/indian-institute-of-technology-guwahati/admission',
   'content': 'Candidates who meet the IIT Guwahati CAT cutoff 2024 can proceed with the MBA admission process. IIT Guwahati Ph.D admissions 2025 are based on the scores of written tests followed by the interview process. The candidates should submit the required documents along with the payment of the admission fee to secure the IIT Guwahati admission.',
   'score': 0.86928266,
   'raw_content': None},
  {'title': 'Indian Institute of Technology Guwahati',
   'url': 'https://www.iitg.ac.in/acad/admission/',
   'content': 'IIT Guwahati offers 4-year Bachelor of Science (BS) in Biomedical Science and Engineering through Jyoti and Bhupat Mehta School of Health Science

now lets define a graph

In [12]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Modification: tell the LLM which tools it can call
# highlight-next-line
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

now we will creaate a function that will call the tools

In [20]:
import json

from langchain_core.messages import ToolMessage


class BasicToolNode:


    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}


tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

Adding conditional edges (they will route flow from one node to other)

In [28]:
def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "END" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", END: END},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

ValueError: Branch with name `route_tools` already exists for node `chatbot`

Now lets ask him basic questions